In [ ]:
# IMPORT DES OUTILS
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

# from scraper_fonctions import defilement, recolte_resultats, click_volet_telechargement, verif_bouton_txt, recolte_metadonnees, recolte_metadonnees_générique, recolte_texte

import time
import webbrowser
import pandas as pd
import requests
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
driver = webdriver.Chrome()
driver.set_window_size(1920, 1080)


In [ ]:
# DÉFINITION FONCTION "defilement"


def defilement():
    # WebDriverWait(driver, 25).until(expected_conditions.element_to_be_clickable((By.XPATH , '//*[@id="idRadioButtonFormat_texteBrut"]')))
    boite_scroll = driver.find_element_by_id("mCSB_1")
    boite_scroll.click()

    # Défilement via simulation touches T inversées (meilleurs résultats qu'avec un défilement souris)
    ActionChains(driver).key_down(Keys.ARROW_DOWN).perform()
    time.sleep(6)
    ActionChains(driver).key_up(Keys.ARROW_DOWN).perform()


# DÉFINITION FONCTION "recolte_resultats"


def recolte_resultats():
    elements_resultats = driver.find_elements_by_xpath(
        "//div[@class='main-infos']/h2/a"
    )
    liens_resultats = []

    for c in elements_resultats:
        liens_resultats.append(c.get_attribute("href"))

    return liens_resultats


# DÉFINITION FONCTION "click_volet_telechargement"


def click_volet_telechargement():

    # bouton_choix_telechargement = driver.find_element_by_xpath("/html/body/main/div[2]/div[3]/div[5]/div/ul/li[3]/span/a/span[1]")
    bouton_choix_telechargement = driver.find_element_by_xpath(
        '//*[@id="pa-icon-download"]/span[2]'
    )
    bouton_choix_telechargement.click()


# DÉFINITION DE LA FONCTION "verif_bouton_txt"


def verif_bouton_txt():

    statut_txt = False

    bouton_choix_txt = '//*[@id="idRadioButtonFormat_texteBrut"]'

    try:
        driver.find_element_by_xpath(bouton_choix_txt)
        statut_txt = True
    except NoSuchElementException:
        statut_txt = False

    return statut_txt


# DÉFINITION DE LA FONCTION création dictionnaire métadonnées + ouverture de la boite + appel pour chaque de la fonction de selection des caractères précis


def recolte_metadonnees():

    metadata = {
        "id": int(),
        "titre": "",
        "auteur": "",
        "date": "",
        "sujet": "",
        "texte": "",
    }

    bouton_telechargement = ("/html/body/main/div[2]/div[3]/div[5]/div/ul/li[3]/span/a/span[1]")

    # OUVERTURE DU MENU "EN SAVOIR PLUS" AVEC ATTENTE ET COLLECTE
    bouton_en_savoir_plus = driver.find_element_by_xpath(
        '//*[@id="acc_1-0_panel_2_trigger"]'
    )
    bouton_en_savoir_plus.click()
    try:
        element = WebDriverWait(driver, 25).until(
            expected_conditions.presence_of_element_located(
                (By.XPATH, bouton_telechargement)
            )
        )

    finally:
        pass

    # PROBLÈME XPATH "sujet" = TOMBE À COTÉ EN FONCTION DES RÉSULTATS
    metadata["titre"] = recolte_metadonnees_générique("Titre")
    metadata["auteur"] = recolte_metadonnees_générique("Auteur")
    metadata["date"] = recolte_metadonnees_générique("Dated'édition")
    metadata["sujet"] = recolte_metadonnees_générique("Sujet")

    return metadata


def recolte_metadonnees_générique(nom_metadonnee):

    elements = driver.find_elements_by_xpath(
        '//*[@id="enSavoirPlusContent"]/div/div/div[1]/div[1]/div'
    )

    i = 0
    emplacement_titre = 0

    for element in elements:
        if (
            element.get_attribute("textContent").replace(" ", "")[
                0 : len(nom_metadonnee)
            ]
            == nom_metadonnee
        ):

            emplacement_titre = i
            # print(emplacement_titre)
            break

        i += 1

    metadata = driver.find_element_by_xpath(
        f'//*[@id="enSavoirPlusContent"]/div/div/div[1]/div[1]/div[{emplacement_titre + 1}]/span[2]'
    )

    return metadata.get_attribute("textContent")


# DÉFINITION DE LA FONCTION "recolte_texte"


def recolte_texte(metadata):

    # OUVERTURE ET COLLECTE DU TEXTE INTÉGRAL

    # WebDriverWait(driver, 25).until(expected_conditions.element_to_be_clickable((By.XPATH , '//*[@id="idRadioButtonFormat_texteBrut"]')))

    bouton_choix_txt = driver.find_element_by_xpath(
        '//*[@id="idRadioButtonFormat_texteBrut"]'
    )
    bouton_choix_txt.click()

    # mettre ici la fonction de défilement
    # click déja effectué plus haut, donc bien sélectionné

    conditions_utilisation = driver.find_element_by_xpath('//*[@id="checkboxes-0"]')
    conditions_utilisation.click()

    bouton_télécharger = driver.find_element_by_xpath(
        '//*[@id="idButtonLancerTelechargement"]'
    )
    bouton_télécharger.click()

    driver.switch_to.window(driver.window_handles[-1])

    # Attente de "En savoir plus sur l'OCR"
    try:
        element = WebDriverWait(driver, 25).until(
            expected_conditions.presence_of_all_elements_located((By.XPATH, "//p"))
        )
    finally:
        pass

    metadata["texte"] = driver.find_element_by_tag_name("html").text

    # Fermeture de l'onglet
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

    return metadata


In [ ]:
# DÉCLARATION DE LA VARIABLE "id" HORS DE TOUTE FONTION POUR INDEXER CHAQUE DOCUMENT

id = 0

# DÉCLARATION DES VARIABLES COMMUNES

sortie = []
nb_doc_ignore_result = 0
nb_doc_ignore_noOcr = 0


In [ ]:
# BOUCLE D'EXPLOITATION DE CHACUN DES RÉSULTATS À PARTIR DE "LIENS_RÉSULTATS"

def exploitation_resultats(entree):
    
    ignore_noOcr = 0
    ignore_result = 0
    problemes = []

    for l in entree:
        driver.get(l)
        try:
            click_volet_telechargement()         
            if verif_bouton_txt() == True:
                dict_metadonnees = recolte_metadonnees()
                dict_metadonnees = recolte_texte(dict_metadonnees)
                time.sleep(1)
                sortie.append(dict_metadonnees)
            else:
                ignore_noOcr += 1
                pass
            time.sleep(1)
        except:
            ignore_result += 1
            pass

    return ignore_noOcr, ignore_result

In [ ]:
%%time

# BOUCLE PAGES

fin_résultats = False
page = 0

while fin_résultats == False:
    
    page += 1
    driver.get(f"https://gallica.bnf.fr/services/engine/search/sru?operation=searchRetrieve&version=1.2&maximumRecords=15&collapsing=true&exactSearch=false&query=gallica%20all%20%22gymnastique%22%20%20and%20%28dc.language%20all%20%22fre%22%29%20and%20%28dc.type%20all%20%22monographie%22%20or%20dc.type%20all%20%22fascicule%22%29%20and%20%28gallicapublication_date%3E%3D%221848%22%20and%20gallicapublication_date%3C%3D%221914%22%29&filter=&page={page}")
    
    try:
        driver.find_element_by_class_name("errorMessage")
        fin_résultats = True
    except:
        
        #try:
            #element = WebDriverWait(driver, 25).until(expected_conditions.presence_of_element_located((By.XPATH , '//*[@id="focusAffinage"]')))
        #except:
            #print(f"La page {page} n'a pas pu charger")
        
        defilement()
        
        log = exploitation_resultats(recolte_resultats())
        
        if page == 2:
            break

In [ ]:
# MESSAGE DE RETOUR DES ERREURS

#bilan_erreurs = f"{log[0]} cas d'OCR absents et {log[1]} cas de docs à résultats multiples."
#print(bilan_erreurs)

In [ ]:
df = pd.DataFrame(sortie)
df.to_csv("data/sortie1_2.csv")

In [ ]:
with open('data/sortie_p1-30.csv', 'a') as f2:
    f2.write('\n')
    f2.write(original)